# Descarga De Datos

In [25]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Asumiendo que min_date_str y max_date_str están disponibles de la ejecución de la celda anterior
# Si no, necesitarías ejecutar la celda anterior nuevamente o establecerlos manualmente
# Nota: Como comentamos la celda anterior, necesitarás definir min_date_str y max_date_str manualmente
# Por ejemplo:
min_date_str = '2015-01-01T00:00:00.000' # Reemplaza con la fecha mínima real si la conoces
max_date_str = '2026-08-09T23:59:59.999' # Reemplaza con la fecha máxima real si la conoces


fecha_min = datetime.fromisoformat(min_date_str.replace('Z', '+00:00')) if 'Z' in min_date_str else datetime.fromisoformat(min_date_str)
fecha_max = datetime.fromisoformat(max_date_str.replace('Z', '+00:00')) if 'Z' in max_date_str else datetime.fromisoformat(max_date_str)

rangos_trimestrales = []
fecha_inicio_actual = fecha_min

while fecha_inicio_actual <= fecha_max:
    fin_de_trimestre = fecha_inicio_actual + relativedelta(months=3) - relativedelta(days=1)
    # Asegurar que la fecha de fin no exceda la fecha_max
    if fin_de_trimestre > fecha_max:
        fin_de_trimestre = fecha_max

    rangos_trimestrales.append((fecha_inicio_actual, fin_de_trimestre))

    fecha_inicio_actual = fecha_inicio_actual + relativedelta(months=3)

# Mostrar los rangos trimestrales generados
for inicio, fin in rangos_trimestrales:
    print(f"Inicio: {inicio.strftime('%Y-%m-%d')}, Fin: {fin.strftime('%Y-%m-%d')}")

Inicio: 2015-01-01, Fin: 2015-03-31
Inicio: 2015-04-01, Fin: 2015-06-30
Inicio: 2015-07-01, Fin: 2015-09-30
Inicio: 2015-10-01, Fin: 2015-12-31
Inicio: 2016-01-01, Fin: 2016-03-31
Inicio: 2016-04-01, Fin: 2016-06-30
Inicio: 2016-07-01, Fin: 2016-09-30
Inicio: 2016-10-01, Fin: 2016-12-31
Inicio: 2017-01-01, Fin: 2017-03-31
Inicio: 2017-04-01, Fin: 2017-06-30
Inicio: 2017-07-01, Fin: 2017-09-30
Inicio: 2017-10-01, Fin: 2017-12-31
Inicio: 2018-01-01, Fin: 2018-03-31
Inicio: 2018-04-01, Fin: 2018-06-30
Inicio: 2018-07-01, Fin: 2018-09-30
Inicio: 2018-10-01, Fin: 2018-12-31
Inicio: 2019-01-01, Fin: 2019-03-31
Inicio: 2019-04-01, Fin: 2019-06-30
Inicio: 2019-07-01, Fin: 2019-09-30
Inicio: 2019-10-01, Fin: 2019-12-31
Inicio: 2020-01-01, Fin: 2020-03-31
Inicio: 2020-04-01, Fin: 2020-06-30
Inicio: 2020-07-01, Fin: 2020-09-30
Inicio: 2020-10-01, Fin: 2020-12-31
Inicio: 2021-01-01, Fin: 2021-03-31
Inicio: 2021-04-01, Fin: 2021-06-30
Inicio: 2021-07-01, Fin: 2021-09-30
Inicio: 2021-10-01, Fin: 202

In [ ]:
import requests
import pandas as pd
from dateutil.relativedelta import relativedelta
import time # Importar time para añadir retrasos
import os # Importar os para manipulación de rutas

# Asegúrate de que Google Drive esté montado antes de ejecutar esta celda
# from google.colab import drive
# drive.mount('/content/drive')


datos_trimestrales_completos = []
url = "https://www.datos.gov.co/resource/p6dx-8zbt.json" # URL corregida con el nuevo dataset ID
directorio_salida = '/content/drive/MyDrive/DatosAPI_Trimestral_Parquet/' # Directorio donde se guardarán los archivos en formato Parquet

# Crear el directorio si no existe
os.makedirs(directorio_salida, exist_ok=True)

# Asumiendo que rangos_trimestrales está disponible de la ejecución de la celda anterior

for fecha_inicio, fecha_fin in rangos_trimestrales:
    fecha_inicio_str = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')
    print(f"Obteniendo datos para el trimestre: {fecha_inicio_str} a {fecha_fin_str}")

    params = {
        "$where": f"fecha_de_publicacion_del >= '{fecha_inicio.strftime('%Y-%m-%dT%H:%M:%S')}' AND fecha_de_publicacion_del <= '{fecha_fin.strftime('%Y-%m-%dT%H:%M:%S')}'",
        "$limit": 5000000 # Límite aumentado, pero aún es bueno manejar la paginación potencial si es necesario
    }

    res = requests.get(url, params=params)

    if res.status_code == 200:
        data = res.json()
        if data: # Procesar solo si hay datos
            df_trimestre = pd.DataFrame(data)
            # Generar nombre del archivo basado en el rango de fechas con extensión .parquet
            nombre_archivo = f"datos_trimestre_{fecha_inicio.year}_Q{(fecha_inicio.month-1)//3 + 1}_{fecha_inicio_str}_to_{fecha_fin_str}.parquet"
            ruta_archivo = os.path.join(directorio_salida, nombre_archivo)
            df_trimestre.to_parquet(ruta_archivo, index=False) # Cambiado a to_parquet
            print(f"Guardados {len(data)} registros en {ruta_archivo}")
        else:
            print(f"No se obtuvieron registros para este trimestre: {fecha_inicio_str} a {fecha_fin_str}.")
    else:
        print(f"Error al obtener datos para {fecha_inicio_str} a {fecha_fin_str}: {res.status_code}")
        print(res.text) # Imprimir respuesta de error para depuración

    time.sleep(1) # Añadir un pequeño retraso para evitar saturar la API

print("Descarga de datos para todos los trimestres finalizada.")

Obteniendo datos para el trimestre: 2015-01-01 a 2015-03-31
No se obtuvieron registros para este trimestre: 2015-01-01 a 2015-03-31.
Obteniendo datos para el trimestre: 2015-04-01 a 2015-06-30
Guardados 313 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q2_2015-04-01_to_2015-06-30.parquet
Obteniendo datos para el trimestre: 2015-07-01 a 2015-09-30
Guardados 2617 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q3_2015-07-01_to_2015-09-30.parquet
Obteniendo datos para el trimestre: 2015-10-01 a 2015-12-31
Guardados 342 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q4_2015-10-01_to_2015-12-31.parquet
Obteniendo datos para el trimestre: 2016-01-01 a 2016-03-31
Guardados 596 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2016_Q1_2016-01-01_to_2016-03-31.parquet
Obteniendo datos para el trimestre: 2016-04-01 a 2016-06-30
Guardados 3002 registros e

In [ ]:
import pandas as pd
import os

output_dir = '/content/drive/MyDrive/DatosAPI_Trimestral_Parquet/' # Directorio donde se guardaron los archivos parquet

# Obtener una lista de todos los archivos parquet en el directorio
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.parquet')]

if all_files:
    print(f"Se encontraron {len(all_files)} archivos Parquet.")
    # Seleccionar algunos archivos para verificar (por ejemplo, el primero, el del medio y el último)
    files_to_verify = []
    if len(all_files) >= 1:
        files_to_verify.append(all_files[0])
    if len(all_files) >= 2:
        files_to_verify.append(all_files[len(all_files) // 2])
    if len(all_files) >= 3:
        files_to_verify.append(all_files[-1])
    # Eliminar duplicados si los hay
    files_to_verify = list(set(files_to_verify))

    # Configurar opciones de pandas para mostrar todas las columnas (opcional)
    pd.set_option('display.max_columns', None)

    for f in files_to_verify:
        print(f"\nVerificando archivo: {f}")
        try:
            df_verify = pd.read_parquet(f) # Cambiado a read_parquet
            print(f"Número de filas: {len(df_verify)}")
            print("Columnas:", df_verify.columns.tolist())
            display(df_verify.head())
        except Exception as e:
            print(f"Error al leer el archivo {f}: {e}")

    # Restablecer opciones de pandas (opcional)
    # pd.reset_option('display.max_columns')

else:
    print("No se encontraron archivos Parquet en el directorio especificado.")

Se encontraron 42 archivos Parquet.

Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2020_Q3_2020-07-01_to_2020-09-30.parquet
Número de filas: 130586
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 'numero_de_lotes', 'estado_del_procedimiento', 'id_estado_del_procedimiento', 'adjudicad

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion_fase_2,fecha_de_publicacion,fecha_de_publicacion_fase,fecha_de_publicacion_fase_1
0,EMPRESA COLOMBIANA DE PETROLEOS,899999068,Distrito Capital de Bogotá,Bogotá,Nacional,Descentralizada,CO1.REQ.1359176,2552824.,709072474,CO1.BDOS.1317971,SUMINISTRO DE LUBRICANTES A NIVEL NACIONAL PAR...,SUMINISTRO DE LUBRICANTES A NIVEL NACIONAL PAR...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,191448166,Contratación régimen especial,Regla aplicable,30,día(s),Bogotá,Departamento de Compras,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.15101505,Abierto,Otro,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,709072474,Presentación de oferta,None,None,None,None,None,None,None,None
1,SUBRED INTEGRADA DE SERVICIOS DE SALUD NORTE E...,900971006,Distrito Capital de Bogotá,No Definido,Territorial,Descentralizada,CO1.REQ.1359690,INVITACIÓN A COTIZAR No 082-2020 S.NTE,702729500,CO1.BDOS.1318397,MATERIAL DE OSTEOSINTESIS,SE REQUIERE CONTRATAR LOS INSUMOS PARA EL SUMI...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,800000000,Contratación régimen especial (con ofertas),Regla aplicable,2,Mes(es),Bogotá,DIRECCIÓN DE CONTRATACIÓN - COMPRAS,0,0,40,0,0,0,0,30,0,Seleccionado,70,Si,CO1.AWD.800423,703801464,Distrito Capital de Bogotá,No Definido,15000000,ANDRES MAURICIO MOJICA ALAVA,SMITH & NEPHEW,900124455,V1.42321500,Cerrado,Suministros,No Definido,"V142321600, V142321700, V142321800, V142321900...",{'url': 'https://community.secop.gov.co/Public...,702729500,Adjudicado,2020-07-10T00:00:00.000,2020-07-10T00:00:00.000,2020-07-10T00:00:00.000,2020-08-11T00:00:00.000,None,None,None,None
2,MUNICIPIO DE SINCELEJO,800104062,Sucre,Sincelejo,Territorial,Descentralizada,CO1.REQ.1359060,CPS 688-2020,706481892,CO1.BDOS.1318058,BRINDAR APOYO AL AREA DE PRESUPUESTO,BRINDAR APOYO AL AREA DE PRESUPUESTO DE LA ALC...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,17800000,Contratación directa,Servicios profesionales y apoyo a la gestión,6,Mes(es),No Aplica,Oficina Jurídica,0,0,0,0,0,0,0,0,0,Seleccionado,70,No,No Adjudicado,710095258,Sucre,Sincelejo,0,No Adjudicado,PAOLA COHEN URZOLA,64581114,V1.80111600,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,706481892,Adjudicado,None,None,None,None,None,None,None,None
3,ALCALDÍA DE POPAYÁN,891580006,Cauca,Popayán,Territorial,Centralizada,CO1.REQ.1351152,MP-SP-CD-008-2020,703038141,CO1.BDOS.1310133,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN A ...,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN A ...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,8685950,Contra


Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q2_2015-04-01_to_2015-06-30.parquet
Número de filas: 313
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 'numero_de_lotes', 'estado_del_procedimiento', 'id_estado_del_procedimiento', 'adjudicado', 'id_adjudicacion', 'codigoproveedor

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion
0,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.501,ISG-0334-2015,700064108,CO1.BDOS.501,PRESTACIÓN DE SERVICIOS,No definido,Presentación de oferta,2015-04-16T00:00:00.000,2015-04-16T00:00:00.000,2015-04-16T00:00:00.000,0,Contratación régimen especial,Regla aplicable,9,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Evaluación,60,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111701,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
1,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.601,ICGH-0330-2015,700064108,CO1.BDOS.601,PRESTACIÓN DE SERVICIOS,APOYAR A LA DIRECCIÓN MEJORAMIENTO CALIDAD DE ...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,10,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
2,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.605,IJ-0354-2015,700064108,CO1.BDOS.606,PRESTACIÓN DE SERVICIOS,PRESTACIÓN DE SERVICIOS PROFESIONALES PARA EJE...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,10,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
3,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.602,IF-0332-2015,700064108,CO1.BDOS.602,PRESTACIÓN DE SERVICIOS,APOYAR A LA DIRECCIÓN DE JURISDICCIÓN COACTIVA...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,11,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://comm


Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2025_Q3_2025-07-01_to_2025-09-30.parquet
Número de filas: 379087
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 'numero_de_lotes', 'estado_del_procedimiento', 'id_estado_del_procedimiento', 'adjudicado', 'id_adjudicacion', 'codigoprovee

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion_fase_2,fecha_de_publicacion
0,HOSPITAL DEPARTAMENTAL MARIA INMACULADA ESE,891180098,Caquetá,No Definido,Nacional,Centralizada,CO1.REQ.8493326,01895-2025,704428127,CO1.BDOS.8343114,DANNA LEANDRA ZAPATA FIERRO,PRESTAR SUS SERVICIOS PERSONALES EN FORMA INDE...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,7590000,Contratación régimen especial,Decreto 092 de 2017,90,día(s),Florencia,OFICINA DE CONTRATACIÓN,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.85101601,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,704428127,Presentación de oferta,None,None,None,None,None,None
1,UNIVERSIDAD DE NARIÑO,800118954,Nariño,Pasto,Territorial,Centralizada,CO1.REQ.8489239,CSU-252131,700358047,CO1.BDOS.8339011,CONTRATO DE PRESTACION DE SERVICIOS,Contratar servicios profesionales de un Médico...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,11880000,Contratación régimen especial,Decreto 092 de 2017,5,Mes(es),Pasto,CENTRO DE ESTUDIOS EN SALUD DE LA UNIVERSIDAD ...,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111600,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700358047,Presentación de oferta,None,None,None,None,None,None
2,UNIVERSIDAD NACIONAL DE COLOMBIA,899999063,Distrito Capital de Bogotá,No Definido,Nacional,Centralizada,CO1.REQ.8491074,UNAL-OPS-2025-2012-164,700698038,CO1.BDOS.8340759,PRESTAR SERVICIOS TÉCNICOS EN LAS ACTIVIDADES ...,PRESTAR SERVICIOS TÉCNICOS EN LAS ACTIVIDADES ...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,12299584,Contratación régimen especial,Decreto 092 de 2017,92,día(s),Bogotá,Bogotá Fac. Artes,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111600,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700698038,Presentación de oferta,None,None,None,None,None,None
3,MUNICIPIO DE ALTAMIRA,891180118,Huila,Altamira,Territorial,Descentralizada,CO1.REQ.8490407,MACPS-065-2025,704897024,CO1.BDOS.8339693,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN PA...,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN PA...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,16380000,Contratación directa,Servicios profesionales y apoyo a la gestión,6,Mes(es),Altamira,MUNICIPIO DE ALTAMIRA,0,0,0,0,0,0,0,0,0,Seleccionado,70,No,No Adjudicado,718480775,Huila,Isnos,0,No Adjudicado,YOIN ALEJANDRO MUÑOZ NAVIA,1084259471,V1.80111620,Abierto,Prest

# Analisis de ultimo año de estudio


1) Unir 4 trimestres y filtrar ≥ 2025-01-01

In [ ]:
import os, glob, pandas as pd
from datetime import datetime

BASE_DIR = "/content/drive/MyDrive/DatosAPI_Trimestral_Parquet/"
# ajusta los cuatro nombres si ya los conoces; o usa un patrón por año/quarter
files = sorted(glob.glob(os.path.join(BASE_DIR, "datos_trimestre_2025_Q*.parquet")))[:4]

df = pd.concat((pd.read_parquet(f) for f in files), ignore_index=True)

# asegurar tipo datetime
for c in ("fecha_de_publicacion_del","fecha_de_ultima_publicaci","fecha_adjudicacion"):
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

cut = pd.Timestamp("2025-01-01")
df_2025p = df[df["fecha_de_publicacion_del"].ge(cut)].copy()

print(len(files), "archivos unidos,", len(df_2025p), "registros ≥ 2025-01-01")


3 archivos unidos, 1434738 registros ≥ 2025-01-01


2) Esquema estrella + unicidad del “proceso”

In [ ]:
# define el orden: qué versión conservar por proceso
df_2025p["orden_ver"] = df_2025p["fecha_de_ultima_publicaci"].fillna(df_2025p["fecha_de_publicacion_del"])
df_last = (df_2025p
           .sort_values(["id_del_proceso","orden_ver"])
           .groupby("id_del_proceso", as_index=False)
           .tail(1)  # última publicación por proceso
           .drop(columns=["orden_ver"]))


Generamos una tabla que elimina los registros con mismo codigo de proceso y  solo mantenemos los que tiene ulitma fehca de publicacion

In [ ]:
num_original_rows = len(df_2025p)
num_deduplicated_rows = len(df_last)
num_removed_rows = num_original_rows - num_deduplicated_rows

print(f"Número de registros originales: {num_original_rows}")
print(f"Número de registros después de la deduplicación: {num_deduplicated_rows}")
print(f"Número de registros eliminados: {num_removed_rows}")

# Para mostrar algunos de los registros eliminados, podemos encontrar los IDs de proceso duplicados
# y mostrar algunas filas del dataframe original que no están en el dataframe deduplicado.

# Encontrar los IDs de proceso que se eliminaron (los que tenían duplicados y no fueron la última versión)
removed_process_ids = df_2025p[~df_2025p['id_del_proceso'].isin(df_last['id_del_proceso'])]['id_del_proceso'].unique()

print(f"\nEjemplos de IDs de proceso con registros eliminados ({len(removed_process_ids)} IDs únicos):")
# Mostrar algunos registros originales para los primeros IDs de proceso eliminados
sample_removed_ids = removed_process_ids[:5] # Mostrar ejemplos para los primeros 5 IDs

for process_id in sample_removed_ids:
    print(f"\nRegistros originales para el proceso ID: {process_id}")
    display(df_2025p[df_2025p['id_del_proceso'] == process_id])

Número de registros originales: 1434738
Número de registros después de la deduplicación: 1335524
Número de registros eliminados: 99214

Ejemplos de IDs de proceso con registros eliminados (0 IDs únicos):


generamos una tabla en la que miramos que columnas difieren en los registros que estamos eliminando


In [ ]:
# Identificar los IDs de proceso que tienen duplicados (es decir, los que se eliminaron)
duplicate_process_ids = df_2025p[df_2025p.duplicated(subset='id_del_proceso', keep=False)]['id_del_proceso'].unique()

print(f"Número de IDs de proceso con registros duplicados: {len(duplicate_process_ids)}")

# Seleccionar algunos ejemplos de IDs de proceso con duplicados para mostrar las diferencias
if len(duplicate_process_ids) > 0:
    # Mostrar ejemplos para los primeros 3 IDs duplicados (puedes ajustar este número)
    sample_duplicate_ids = duplicate_process_ids[:3]

    for sample_process_id in sample_duplicate_ids:
        print(f"\nMostrando diferencias para un ID de proceso de ejemplo: {sample_process_id}")

        # Filtrar el dataframe original para el ID de proceso de ejemplo
        df_original_process = df_2025p[df_2025p['id_del_proceso'] == sample_process_id].copy()

        # Convertir columnas de tipo diccionario o lista a string para permitir la comparación
        for col in df_original_process.columns:
            if df_original_process[col].apply(lambda x: isinstance(x, (dict, list))).any():
                 df_original_process[col] = df_original_process[col].astype(str)

        # Encontrar las columnas que tienen valores diferentes en los registros de este proceso
        diff_cols = []
        if len(df_original_process) > 1:
            first_row = df_original_process.iloc[0]
            for col in df_original_process.columns:
                # Usamos .nunique() para verificar si hay más de un valor único en la columna para este grupo
                if df_original_process[col].nunique() > 1:
                    diff_cols.append(col)

        if diff_cols:
            print(f"\nColumnas que difieren para el proceso ID {sample_process_id}: {diff_cols}")
            print("\nRegistros originales para este proceso, mostrando todas las columnas:")
            # Mostrar todas las columnas para estos registros duplicados
            display(df_2025p[df_2025p['id_del_proceso'] == sample_process_id])
        else:
            print("\nNo se encontraron columnas que difieran en los registros de este proceso de ejemplo.")

else:
    print("\nNo se encontraron IDs de proceso con registros duplicados en df_2025p.")

Número de IDs de proceso con registros duplicados: 19681

Mostrando diferencias para un ID de proceso de ejemplo: CO1.REQ.7382459

Columnas que difieren para el proceso ID CO1.REQ.7382459: ['codigoproveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este proceso, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
15,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7382459,RE-CD-103-2025,703827568,CO1.BDOS.7246606,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01
1676,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7382459,RE-CD-103-2025,703827568,CO1.BDOS.7246606,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01



Mostrando diferencias para un ID de proceso de ejemplo: CO1.REQ.7383534

Columnas que difieren para el proceso ID CO1.REQ.7383534: ['codigoproveedor', 'departamento_proveedor', 'ciudad_proveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este proceso, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
78,E.S.E HOSPITAL SAN RAFAEL DE CAQUEZA,832001411,Cundinamarca,Caqueza,Territorial,Descentralizada,CO1.REQ.7383534,HSRC-014-2025,703015735,CO1.BDOS.7247233,...,{'url': 'https://community.secop.gov.co/Public...,703015735,Adjudicado,None,None,None,None,NaT,None,2025-01-01
896,E.S.E HOSPITAL SAN RAFAEL DE CAQUEZA,832001411,Cundinamarca,Caqueza,Territorial,Descentralizada,CO1.REQ.7383534,HSRC-014-2025,703015735,CO1.BDOS.7247233,...,{'url': 'https://community.secop.gov.co/Public...,703015735,Adjudicado,None,None,None,None,NaT,None,2025-01-01



Mostrando diferencias para un ID de proceso de ejemplo: CO1.REQ.7383611

Columnas que difieren para el proceso ID CO1.REQ.7383611: ['codigoproveedor', 'departamento_proveedor', 'ciudad_proveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este proceso, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
104,ALEJANDRO PROSPERO REVEREND E.S.E.,819004070,Magdalena,Santa Marta,Territorial,Descentralizada,CO1.REQ.7383611,PS-127-2025,704290097,CO1.BDOS.7247315,...,{'url': 'https://community.secop.gov.co/Public...,704290097,Adjudicado,None,None,None,None,NaT,None,2025-01-01
743,ALEJANDRO PROSPERO REVEREND E.S.E.,819004070,Magdalena,Santa Marta,Territorial,Descentralizada,CO1.REQ.7383611,PS-127-2025,704290097,CO1.BDOS.7247315,...,{'url': 'https://community.secop.gov.co/Public...,704290097,Adjudicado,None,None,None,None,NaT,None,2025-01-01


Si se necesita nivel de contrato adjudicado: hay procesos con varias adjudicaciones o lotes. Cambio del grano a (id_del_proceso, id_adjudicacion) y deduplica por ese par:

In [ ]:
if "id_adjudicacion" in df_2025p.columns:
    keys = ["id_del_proceso","id_adjudicacion"]
    df_2025p["orden_ver"] = df_2025p["fecha_de_ultima_publicaci"].fillna(df_2025p["fecha_de_publicacion_del"])
    df_last = (df_2025p
               .sort_values(keys+["orden_ver"])
               .groupby(keys, as_index=False)
               .tail(1)
               .drop(columns=["orden_ver"]))


In [ ]:
# Identificar los pares (id_del_proceso, id_adjudicacion) que tienen duplicados en df_2025p
if "id_adjudicacion" in df_2025p.columns:
    duplicate_adjudication_keys = df_2025p[df_2025p.duplicated(subset=["id_del_proceso", "id_adjudicacion"], keep=False)][["id_del_proceso", "id_adjudicacion"]].drop_duplicates().values.tolist()

    print(f"Número de pares (Proceso, Adjudicación) con registros duplicados: {len(duplicate_adjudication_keys)}")

    # Seleccionar algunos ejemplos de pares duplicados para mostrar las diferencias
    if len(duplicate_adjudication_keys) > 0:
        # Mostrar ejemplos para los primeros 3 pares duplicados (puedes ajustar este número)
        sample_duplicate_keys = duplicate_adjudication_keys[:3]

        for process_id, adjudicacion_id in sample_duplicate_keys:
            print(f"\nMostrando diferencias para el par (Proceso, Adjudicación): ({process_id}, {adjudicacion_id})")

            # Filtrar el dataframe original para el par de ejemplo
            df_original_pair = df_2025p[(df_2025p['id_del_proceso'] == process_id) & (df_2025p['id_adjudicacion'] == adjudicacion_id)].copy()

            # Convertir columnas de tipo diccionario o lista a string para permitir la comparación
            for col in df_original_pair.columns:
                if df_original_pair[col].apply(lambda x: isinstance(x, (dict, list))).any():
                     df_original_pair[col] = df_original_pair[col].astype(str)

            # Encontrar las columnas que tienen valores diferentes en los registros de este par
            diff_cols = []
            if len(df_original_pair) > 1:
                first_row = df_original_pair.iloc[0]
                for col in df_original_pair.columns:
                    if df_original_pair[col].nunique() > 1:
                        diff_cols.append(col)

            if diff_cols:
                print(f"\nColumnas que difieren para el par ({process_id}, {adjudicacion_id}): {diff_cols}")
                print("\nRegistros originales para este par, mostrando todas las columnas:")
                # Mostrar todas las columnas para estos registros duplicados
                display(df_original_pair)
            else:
                print("\nNo se encontraron columnas que difieran en los registros de este par de ejemplo.")

    else:
        print("\nNo se encontraron pares (Proceso, Adjudicación) con registros duplicados.")
else:
    print("\nLa columna 'id_adjudicacion' no existe en el DataFrame.")

Número de pares (Proceso, Adjudicación) con registros duplicados: 19137

Mostrando diferencias para el par (Proceso, Adjudicación): (CO1.REQ.7382459, No Adjudicado)

Columnas que difieren para el par (CO1.REQ.7382459, No Adjudicado): ['codigoproveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este par, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
15,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7382459,RE-CD-103-2025,703827568,CO1.BDOS.7246606,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01
1676,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7382459,RE-CD-103-2025,703827568,CO1.BDOS.7246606,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01



Mostrando diferencias para el par (Proceso, Adjudicación): (CO1.REQ.7383534, No Adjudicado)

Columnas que difieren para el par (CO1.REQ.7383534, No Adjudicado): ['codigoproveedor', 'departamento_proveedor', 'ciudad_proveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este par, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
78,E.S.E HOSPITAL SAN RAFAEL DE CAQUEZA,832001411,Cundinamarca,Caqueza,Territorial,Descentralizada,CO1.REQ.7383534,HSRC-014-2025,703015735,CO1.BDOS.7247233,...,{'url': 'https://community.secop.gov.co/Public...,703015735,Adjudicado,None,None,None,None,NaT,None,2025-01-01
896,E.S.E HOSPITAL SAN RAFAEL DE CAQUEZA,832001411,Cundinamarca,Caqueza,Territorial,Descentralizada,CO1.REQ.7383534,HSRC-014-2025,703015735,CO1.BDOS.7247233,...,{'url': 'https://community.secop.gov.co/Public...,703015735,Adjudicado,None,None,None,None,NaT,None,2025-01-01



Mostrando diferencias para el par (Proceso, Adjudicación): (CO1.REQ.7383611, No Adjudicado)

Columnas que difieren para el par (CO1.REQ.7383611, No Adjudicado): ['codigoproveedor', 'departamento_proveedor', 'ciudad_proveedor', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado']

Registros originales para este par, mostrando todas las columnas:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
104,ALEJANDRO PROSPERO REVEREND E.S.E.,819004070,Magdalena,Santa Marta,Territorial,Descentralizada,CO1.REQ.7383611,PS-127-2025,704290097,CO1.BDOS.7247315,...,{'url': 'https://community.secop.gov.co/Public...,704290097,Adjudicado,None,None,None,None,NaT,None,2025-01-01
743,ALEJANDRO PROSPERO REVEREND E.S.E.,819004070,Magdalena,Santa Marta,Territorial,Descentralizada,CO1.REQ.7383611,PS-127-2025,704290097,CO1.BDOS.7247315,...,{'url': 'https://community.secop.gov.co/Public...,704290097,Adjudicado,None,None,None,None,NaT,None,2025-01-01


In [ ]:
# Records eliminated in the first deduplication (by id_del_proceso)
# We need to re-create the df_last from the first deduplication step
df_last_process_only = (df_2025p
           .sort_values(["id_del_proceso","orden_ver"])
           .groupby("id_del_proceso", as_index=False)
           .tail(1))

eliminated_in_process_only_dedup = df_2025p[~df_2025p.index.isin(df_last_process_only.index)].copy()

# Records eliminated in the second deduplication (by id_del_proceso and id_adjudicacion)
# This is the df_last we currently have
eliminated_in_process_adjudication_dedup = df_2025p[~df_2025p.index.isin(df_last.index)].copy()

print(f"Número de registros eliminados en la primera deduplicación (solo por proceso): {len(eliminated_in_process_only_dedup)}")
print(f"Número de registros eliminados en la segunda deduplicación (por proceso y adjudicación): {len(eliminated_in_process_adjudication_dedup)}")

# Find records eliminated in the first but NOT the second
eliminated_only_in_process_dedup = eliminated_in_process_only_dedup[
    ~eliminated_in_process_only_dedup.index.isin(eliminated_in_process_adjudication_dedup.index)
]

# Find records eliminated in the second but NOT the first
eliminated_only_in_process_adjudication_dedup = eliminated_in_process_adjudication_dedup[
    ~eliminated_in_process_adjudication_dedup.index.isin(eliminated_in_process_only_dedup.index)
]

# Find records eliminated in BOTH deduplications
eliminated_in_both_dedups = eliminated_in_process_only_dedup[
    eliminated_in_process_only_dedup.index.isin(eliminated_in_process_adjudication_dedup.index)
]

print(f"\nRegistros eliminados solo en la primera deduplicación: {len(eliminated_only_in_process_dedup)}")
print(f"Registros eliminados solo en la segunda deduplicación: {len(eliminated_only_in_process_adjudication_dedup)}")
print(f"Registros eliminados en ambas deduplicaciones: {len(eliminated_in_both_dedups)}")

# Optionally, display some examples from each category
if len(eliminated_only_in_process_dedup) > 0:
    print("\nEjemplo de registros eliminados solo en la primera deduplicación:")
    display(eliminated_only_in_process_dedup.head())

if len(eliminated_only_in_process_adjudication_dedup) > 0:
    print("\nEjemplo de registros eliminados solo en la segunda deduplicación:")
    display(eliminated_only_in_process_adjudication_dedup.head())

if len(eliminated_in_both_dedups) > 0:
    print("\nEjemplo de registros eliminados en ambas deduplicaciones:")
    display(eliminated_in_both_dedups.head())

Número de registros eliminados en la primera deduplicación (solo por proceso): 99214
Número de registros eliminados en la segunda deduplicación (por proceso y adjudicación): 95563

Registros eliminados solo en la primera deduplicación: 3651
Registros eliminados solo en la segunda deduplicación: 0
Registros eliminados en ambas deduplicaciones: 95563

Ejemplo de registros eliminados solo en la primera deduplicación:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
6575,HOSPITAL SAN JUAN DE DIOS ESE RIONEGRO,890907254,Antioquia,Rionegro,Territorial,Descentralizada,CO1.REQ.7391781,INVITACION CERRADA N° 001-2025,701728222,CO1.BDOS.7255940,...,{'url': 'https://community.secop.gov.co/Public...,701728222,Adjudicado,None,2025-01-14T00:00:00.000,2025-01-14T00:00:00.000,2025-01-15T00:00:00.000,NaT,None,2025-01-03
17095,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION DE R...,900498879,Distrito Capital de Bogotá,Bogotá,Nacional,Centralizada,CO1.REQ.7409734,IP-URT-5-2025,700447030,CO1.BDOS.7273626,...,{'url': 'https://community.secop.gov.co/Public...,700447030,Adjudicado,None,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-17,None,2025-01-07
17512,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION DE R...,900498879,Distrito Capital de Bogotá,Bogotá,Nacional,Centralizada,CO1.REQ.7410140,IP-URT-6-2025,700447030,CO1.BDOS.7274025,...,{'url': 'https://community.secop.gov.co/Public...,700447030,Adjudicado,None,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-17,None,2025-01-07
19522,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION DE R...,900498879,Distrito Capital de Bogotá,Bogotá,Nacional,Centralizada,CO1.REQ.7410427,IP-URT-7-2025,700447030,CO1.BDOS.7274314,...,{'url': 'https://community.secop.gov.co/Public...,700447030,Adjudicado,None,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-17,None,2025-01-07
19784,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION DE R...,900498879,Distrito Capital de Bogotá,Bogotá,Nacional,Centralizada,CO1.REQ.7408977,IP-URT-4-2025,700447030,CO1.BDOS.7272878,...,{'url': 'https://community.secop.gov.co/Public...,700447030,Adjudicado,None,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-13T00:00:00.000,2025-01-17,None,2025-01-07



Ejemplo de registros eliminados en ambas deduplicaciones:


,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,...,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion,orden_ver
15,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7382459,RE-CD-103-2025,703827568,CO1.BDOS.7246606,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01
78,E.S.E HOSPITAL SAN RAFAEL DE CAQUEZA,832001411,Cundinamarca,Caqueza,Territorial,Descentralizada,CO1.REQ.7383534,HSRC-014-2025,703015735,CO1.BDOS.7247233,...,{'url': 'https://community.secop.gov.co/Public...,703015735,Adjudicado,None,None,None,None,NaT,None,2025-01-01
104,ALEJANDRO PROSPERO REVEREND E.S.E.,819004070,Magdalena,Santa Marta,Territorial,Descentralizada,CO1.REQ.7383611,PS-127-2025,704290097,CO1.BDOS.7247315,...,{'url': 'https://community.secop.gov.co/Public...,704290097,Adjudicado,None,None,None,None,NaT,None,2025-01-01
149,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7384357,RE-CD-010-2025,703827568,CO1.BDOS.7247951,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01
152,EMPRESA SOCIAL DEL ESTADO HOSPITAL SAN RAFAEL ...,891800231,Boyacá,Tunja,Territorial,Descentralizada,CO1.REQ.7384602,RE-CD-180-2025,703827568,CO1.BDOS.7247698,...,{'url': 'https://community.secop.gov.co/Public...,703827568,Adjudicado,None,None,None,None,NaT,None,2025-01-01


3) Construcción de dimensiones clave para el esquemas estrella

In [ ]:
import numpy as np

# --- Dim Entidad (compradora) ---
cols_ent = ["codigo_entidad","entidad","ordenentidad","codigo_pci","nombre_de_la_unidad_de",
            "departamento_entidad","ciudad_entidad"]
D_Entidad = (df_last[cols_ent]
             .drop_duplicates()
             .reset_index(drop=True))
D_Entidad.insert(0, "entidad_id", np.arange(1, len(D_Entidad)+1))

# --- Dim Proveedor (si aplica) ---
if "nit_del_proveedor_adjudicado" in df_last.columns:
    cols_prov = ["nit_del_proveedor_adjudicado","nombre_del_proveedor","departamento_proveedor","ciudad_proveedor"]
    D_Proveedor = (df_last[cols_prov]
                   .rename(columns={"nit_del_proveedor_adjudicado":"nit"})
                   .drop_duplicates()
                   .reset_index(drop=True))
    D_Proveedor.insert(0, "proveedor_id", np.arange(1, len(D_Proveedor)+1))

# --- Dim Tiempo (publicación) ---
tpub = df_last["fecha_de_publicacion_del"].dropna().dt.normalize().drop_duplicates().to_frame(name="fecha")
D_Tiempo = tpub.sort_values("fecha").reset_index(drop=True)
D_Tiempo.insert(0, "tiempo_id", np.arange(1, len(D_Tiempo)+1))
D_Tiempo["año"] = D_Tiempo["fecha"].dt.year
D_Tiempo["trimestre"] = ((D_Tiempo["fecha"].dt.month-1)//3 + 1)
D_Tiempo["mes"] = D_Tiempo["fecha"].dt.month
D_Tiempo["día"] = D_Tiempo["fecha"].dt.day

# --- Dim Ubicaciones (entidad y proveedor) ---
D_UbiEntidad = (df_last[["departamento_entidad","ciudad_entidad"]]
                .drop_duplicates().reset_index(drop=True))
D_UbiEntidad.insert(0, "ubi_entidad_id", np.arange(1, len(D_UbiEntidad)+1))

D_UbiProveedor = (df_last[["departamento_proveedor","ciudad_proveedor"]]
                  .drop_duplicates().reset_index(drop=True))
D_UbiProveedor.insert(0, "ubi_proveedor_id", np.arange(1, len(D_UbiProveedor)+1))

# --- Dim Categoría / Modalidad / TipoContrato ---
D_Categoria = (df_last[["codigo_principal_de_categoria"]].drop_duplicates().reset_index(drop=True))
D_Categoria.insert(0, "categoria_id", np.arange(1, len(D_Categoria)+1))

D_Modalidad = (df_last[["modalidad_de_contratacion","justificaci_n_modalidad_de"]]
               .drop_duplicates().reset_index(drop=True))
D_Modalidad.insert(0, "modalidad_id", np.arange(1, len(D_Modalidad)+1))

D_TipoContrato = (df_last[["tipo_de_contrato","subtipo_de_contrato"]]
                  .drop_duplicates().reset_index(drop=True))
D_TipoContrato.insert(0, "tipo_contrato_id", np.arange(1, len(D_TipoContrato)+1))


In [ ]:
# helpers de mapeo
def map_fk(df_src, D, left_cols, right_cols, fk_name):
    tmp = D[right_cols + [D.columns[0]]].copy()  # id en la primera col
    return df_src.merge(tmp, how="left", left_on=left_cols, right_on=right_cols).rename(columns={D.columns[0]: fk_name})

F = df_last.copy()

F = map_fk(F, D_Entidad, ["codigo_entidad","entidad","ordenentidad","codigo_pci","nombre_de_la_unidad_de",
                           "departamento_entidad","ciudad_entidad"],
                          ["codigo_entidad","entidad","ordenentidad","codigo_pci","nombre_de_la_unidad_de",
                           "departamento_entidad","ciudad_entidad"], "entidad_id")

if "nit_del_proveedor_adjudicado" in F.columns:
    F = map_fk(F, D_Proveedor, ["nit_del_proveedor_adjudicado","nombre_del_proveedor","departamento_proveedor","ciudad_proveedor"],
                           ["nit","nombre_del_proveedor","departamento_proveedor","ciudad_proveedor"], "proveedor_id")

F["fecha_pub_norm"] = F["fecha_de_publicacion_del"].dt.normalize()
F = map_fk(F, D_Tiempo, ["fecha_pub_norm"], ["fecha"], "tiempo_id")
F = map_fk(F, D_UbiEntidad, ["departamento_entidad","ciudad_entidad"], ["departamento_entidad","ciudad_entidad"], "ubi_entidad_id")
F = map_fk(F, D_UbiProveedor, ["departamento_proveedor","ciudad_proveedor"], ["departamento_proveedor","ciudad_proveedor"], "ubi_proveedor_id")
F = map_fk(F, D_Categoria, ["codigo_principal_de_categoria"], ["codigo_principal_de_categoria"], "categoria_id")
F = map_fk(F, D_Modalidad, ["modalidad_de_contratacion","justificaci_n_modalidad_de"], ["modalidad_de_contratacion","justificaci_n_modalidad_de"], "modalidad_id")
F = map_fk(F, D_TipoContrato, ["tipo_de_contrato","subtipo_de_contrato"], ["tipo_de_contrato","subtipo_de_contrato"], "tipo_contrato_id")

# columnas finales del hecho
fact_cols = ["id_del_proceso","id_adjudicacion","precio_base","valor_total_adjudicacion","numero_de_lotes",
             "proveedores_invitados","respuestas_al_procedimiento","visualizaciones_del",
             "entidad_id","proveedor_id","tiempo_id","ubi_entidad_id","ubi_proveedor_id",
             "categoria_id","modalidad_id","tipo_contrato_id","estado_del_procedimiento","id_estado_del_procedimiento"]
F_Proceso = F[[c for c in fact_cols if c in F.columns]].copy()


validación adicional de unicidad:

In [ ]:
dup = F_Proceso.duplicated(subset=["id_del_proceso"], keep=False).sum()
print("Procesos no únicos en el hecho (por id_del_proceso):", dup)


Procesos no únicos en el hecho (por id_del_proceso): 6296


### Tabla de Hechos (F_Proceso)

In [ ]:
print("Columnas de la Tabla de Hechos:")
print(F_Proceso.columns.tolist())
print("\nPrimeras 5 filas de la Tabla de Hechos:")
display(F_Proceso.head())

Columnas de la Tabla de Hechos:
['id_del_proceso', 'id_adjudicacion', 'precio_base', 'valor_total_adjudicacion', 'numero_de_lotes', 'proveedores_invitados', 'respuestas_al_procedimiento', 'visualizaciones_del', 'entidad_id', 'proveedor_id', 'tiempo_id', 'ubi_entidad_id', 'ubi_proveedor_id', 'categoria_id', 'modalidad_id', 'tipo_contrato_id', 'estado_del_procedimiento', 'id_estado_del_procedimiento']

Primeras 5 filas de la Tabla de Hechos:


,id_del_proceso,id_adjudicacion,precio_base,valor_total_adjudicacion,numero_de_lotes,proveedores_invitados,respuestas_al_procedimiento,visualizaciones_del,entidad_id,proveedor_id,tiempo_id,ubi_entidad_id,ubi_proveedor_id,categoria_id,modalidad_id,tipo_contrato_id,estado_del_procedimiento,id_estado_del_procedimiento
0,CO1.REQ.1456251,No Adjudicado,96000000,0,0,0,0,0,1,1,91,1,1,1,1,1,Publicado,50
1,CO1.REQ.2520241,No Adjudicado,90000000,0,0,0,0,0,2,1,55,2,1,2,2,2,Publicado,50
2,CO1.REQ.3268070,No Adjudicado,2856000,0,0,0,0,0,3,1,219,3,1,3,2,2,Publicado,50
3,CO1.REQ.3437388,No Adjudicado,6914000,0,0,0,0,0,4,1,225,4,1,4,3,3,Publicado,50
4,CO1.REQ.3437587,No Adjudicado,6914000,0,0,0,0,0,4,1,225,4,1,4,3,3,Publicado,50


### Dimensión Entidad (D_Entidad)

In [ ]:
print("Columnas de la Dimensión Entidad:")
print(D_Entidad.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Entidad:")
display(D_Entidad.head())

Columnas de la Dimensión Entidad:
['entidad_id', 'codigo_entidad', 'entidad', 'ordenentidad', 'codigo_pci', 'nombre_de_la_unidad_de', 'departamento_entidad', 'ciudad_entidad']

Primeras 5 filas de la Dimensión Entidad:


,entidad_id,codigo_entidad,entidad,ordenentidad,codigo_pci,nombre_de_la_unidad_de,departamento_entidad,ciudad_entidad
0,1,700296015,MINCIENCIAS,Nacional,Centralizada,Dirección De Gestión De Recursos Para La Ctel,Distrito Capital de Bogotá,Bogotá
1,2,712182070,EMPRESA SOCIAL DEL ESTADO (ESE) HOSPITAL DEPAR...,Territorial,Descentralizada,"ESE Hospital Departamental de San Andrés, Prov...","San Andrés, Providencia y Santa Catalina",San Andrés
2,3,717972228,Institucion Educativa Departamental JOAQUIN SA...,Territorial,Descentralizada,RECTORIA,Cundinamarca,Quipile
3,4,710346917,POLITECNICO COLOMBIANO JAIME ISAZA CADAVID,Territorial,Descentralizada,EXTENSION,Antioquia,Medellín
4,5,704079805,INSTITUCIÓN EDUCATIVA DEPARTAMENTAL AGROPECUAR...,Territorial,Descentralizada,URIEL AVILA SALAMANCA,Cundinamarca,Quipile


### Dimensión Proveedor (D_Proveedor - si existe)

In [ ]:
if 'D_Proveedor' in locals():
    print("Columnas de la Dimensión Proveedor:")
    print(D_Proveedor.columns.tolist())
    print("\nPrimeras 5 filas de la Dimensión Proveedor:")
    display(D_Proveedor.head())
else:
    print("La Dimensión Proveedor no fue creada porque la columna 'nit_del_proveedor_adjudicado' no existe en los datos.")

Columnas de la Dimensión Proveedor:
['proveedor_id', 'nit', 'nombre_del_proveedor', 'departamento_proveedor', 'ciudad_proveedor']

Primeras 5 filas de la Dimensión Proveedor:


,proveedor_id,nit,nombre_del_proveedor,departamento_proveedor,ciudad_proveedor
0,1,No Definido,No Definido,No Definido,No Definido
1,2,900557908,COLMENARES Y ASOCIADOS ABOGADOS,Distrito Capital de Bogotá,Bogotá
2,3,901492178,ARQUINPRO CONSTRUCCIONES Y ASESORIAS S.A.S.,Cundinamarca,Pacho
3,4,901438622,CORPORACION JUNTOS ES MAS FACIL,Antioquia,Itagui
4,5,1049533351,Alberto Jose Vega Fernandez,Antioquia,Apartadó


### Dimensión Tiempo (D_Tiempo)

In [ ]:
print("Columnas de la Dimensión Tiempo:")
print(D_Tiempo.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Tiempo:")
display(D_Tiempo.head())

Columnas de la Dimensión Tiempo:
['tiempo_id', 'fecha', 'año', 'trimestre', 'mes', 'día']

Primeras 5 filas de la Dimensión Tiempo:


,tiempo_id,fecha,año,trimestre,mes,día
0,1,2025-01-01,2025,1,1,1
1,2,2025-01-02,2025,1,1,2
2,3,2025-01-03,2025,1,1,3
3,4,2025-01-04,2025,1,1,4
4,5,2025-01-05,2025,1,1,5


### Dimensión Ubicación Entidad (D_UbiEntidad)

In [ ]:
print("Columnas de la Dimensión Ubicación Entidad:")
print(D_UbiEntidad.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Ubicación Entidad:")
display(D_UbiEntidad.head())

Columnas de la Dimensión Ubicación Entidad:
['ubi_entidad_id', 'departamento_entidad', 'ciudad_entidad']

Primeras 5 filas de la Dimensión Ubicación Entidad:


,ubi_entidad_id,departamento_entidad,ciudad_entidad
0,1,Distrito Capital de Bogotá,Bogotá
1,2,"San Andrés, Providencia y Santa Catalina",San Andrés
2,3,Cundinamarca,Quipile
3,4,Antioquia,Medellín
4,5,Distrito Capital de Bogotá,No Definido


### Dimensión Ubicación Proveedor (D_UbiProveedor)

In [ ]:
print("Columnas de la Dimensión Ubicación Proveedor:")
print(D_UbiProveedor.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Ubicación Proveedor:")
display(D_UbiProveedor.head())

Columnas de la Dimensión Ubicación Proveedor:
['ubi_proveedor_id', 'departamento_proveedor', 'ciudad_proveedor']

Primeras 5 filas de la Dimensión Ubicación Proveedor:


,ubi_proveedor_id,departamento_proveedor,ciudad_proveedor
0,1,No Definido,No Definido
1,2,Distrito Capital de Bogotá,Bogotá
2,3,Cundinamarca,Pacho
3,4,Antioquia,Itagui
4,5,Antioquia,Apartadó


### Dimensión Categoría (D_Categoria)

In [ ]:
print("Columnas de la Dimensión Categoría:")
print(D_Categoria.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Categoría:")
display(D_Categoria.head())

Columnas de la Dimensión Categoría:
['categoria_id', 'codigo_principal_de_categoria']

Primeras 5 filas de la Dimensión Categoría:


,categoria_id,codigo_principal_de_categoria
0,1,V1.86101600
1,2,V1.85101600
2,3,V1.84131516
3,4,V1.80101604
4,5,V1.45111600


### Dimensión Modalidad (D_Modalidad)

In [ ]:
print("Columnas de la Dimensión Modalidad:")
print(D_Modalidad.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Modalidad:")
display(D_Modalidad.head())

Columnas de la Dimensión Modalidad:
['modalidad_id', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de']

Primeras 5 filas de la Dimensión Modalidad:


,modalidad_id,modalidad_de_contratacion,justificaci_n_modalidad_de
0,1,Contratación régimen especial,Regla aplicable
1,2,Contratación régimen especial,Decreto 092 de 2017
2,3,Contratación directa,Servicios profesionales y apoyo a la gestión
3,4,Contratación directa,Arrendamiento de inmuebles
4,5,Contratación Directa (con ofertas),Adquisición de inmuebles


### Dimensión Tipo Contrato (D_TipoContrato)

In [ ]:
print("Columnas de la Dimensión Tipo Contrato:")
print(D_TipoContrato.columns.tolist())
print("\nPrimeras 5 filas de la Dimensión Tipo Contrato:")
display(D_TipoContrato.head())

Columnas de la Dimensión Tipo Contrato:
['tipo_contrato_id', 'tipo_de_contrato', 'subtipo_de_contrato']

Primeras 5 filas de la Dimensión Tipo Contrato:


,tipo_contrato_id,tipo_de_contrato,subtipo_de_contrato
0,1,Otro,No Definido
1,2,Decreto 092 de 2017,No Definido
2,3,Prestación de servicios,No Definido
3,4,Obra,No Definido
4,5,Suministros,No Definido
